In [34]:
# Maximizing Damage Output for a D&D character 

# There is much variablility in D&D, so we decided to control many variables that can be adjusted.
# The assumed Race is Human and any additional feats are assumed to be Ability Score Increase (ASI)
# To that end, the assumed stats of the characters is 18 in the main stat (Str or Dex), then 16 and 14
# Armor class for enemies has been set to 13. Chance to hit = 1 - (AC-bonus_to_hit)/20
# Reminder: Crits only double damage on dice, not the modifiers.

import cvxpy as cp

enemyAC = 13
strengthScore = 18
proficiency_bonus = 3
# Assume light leather armor
# Assume soldier background

attributes = cp.Variable(6, integer=True) 
racial_bonus = cp.Variable(6, integer=True)
feat_bonus = cp.Variable(6, integer=True)
avg_weapon_dmg = cp.Variable(1, integer=True)
has_proficiency = cp.Variable(18, integer=True)

constraints = []

# Ability Score Handling
constraints.append(cp.sum(feat_bonus) == 2)
constraints.append(cp.sum(attributes) == 72)
    
# Ability Score constraints 8 <= ability <= 18
for i in range(6):
    constraints.append(attributes[i] >= 8)
    constraints.append(attributes[i] <= 18)
    constraints.append(attributes[i] + racial_bonus[i]  + feat_bonus[i] <= 18)

constraints.append(attributes[0] == strengthScore)

# Nonnegativity constraints
for i in range(6):
    constraints.append(attributes[i] >= 0)
    constraints.append(racial_bonus[i] >= 0)
    constraints.append(feat_bonus[i] >= 0)

#Temporarily set all racial bonuses to 0
for i in range(6):
    constraints.append(racial_bonus[i] == 0)

# Weapon damage handling
constraints.append(avg_weapon_dmg[0] >= 2)
constraints.append(avg_weapon_dmg[0] <= 6)

# Weapon damage handling
constraints.append(avg_weapon_dmg[0] >= 2)
constraints.append(avg_weapon_dmg[0] <= 6)

# Proficiency Bonuses
#Choose two skills from Acrobatics, Animal Handling, Athletics, History, Insight, Intimidation, Perception, and Survival
constraints.append(has_proficiency[0] + has_proficiency[1] + has_proficiency[3] + has_proficiency[5] + has_proficiency[6] + has_proficiency[7] + has_proficiency[11] + has_proficiency[17] == 4)
#From soldier background
constraints.append(has_proficiency[3] == 1)
constraints.append(has_proficiency[7] == 1)
#All others zero
constraints.append(has_proficiency[2] == 0)
constraints.append(has_proficiency[4] == 0)
constraints.append(has_proficiency[8] == 0)
constraints.append(has_proficiency[9] == 0)
constraints.append(has_proficiency[10] == 0)
constraints.append(has_proficiency[12] == 0)
constraints.append(has_proficiency[13] == 0)
constraints.append(has_proficiency[14] == 0)
constraints.append(has_proficiency[15] == 0)
constraints.append(has_proficiency[16] == 0)

#Constraints of proficiency bonus
constraints.append(has_proficiency[0] >= 0)
constraints.append(has_proficiency[0] >= 0)
constraints.append(has_proficiency[1] >= 0)
constraints.append(has_proficiency[2] >= 0)
constraints.append(has_proficiency[3] >= 0)
constraints.append(has_proficiency[4] >= 0)
constraints.append(has_proficiency[5] >= 0)
constraints.append(has_proficiency[6] >= 0)
constraints.append(has_proficiency[7] >= 0)
constraints.append(has_proficiency[8] >= 0)
constraints.append(has_proficiency[9] >= 0)
constraints.append(has_proficiency[10] >= 0)
constraints.append(has_proficiency[11] >= 0)
constraints.append(has_proficiency[12] >= 0)
constraints.append(has_proficiency[13] >= 0)
constraints.append(has_proficiency[14] >= 0)
constraints.append(has_proficiency[15] >= 0)
constraints.append(has_proficiency[16] >= 0)
constraints.append(has_proficiency[17] >= 0)
constraints.append(has_proficiency[0] <= 1)
constraints.append(has_proficiency[1] <= 1)
constraints.append(has_proficiency[2] <= 1)
constraints.append(has_proficiency[3] <= 1)
constraints.append(has_proficiency[4] <= 1)
constraints.append(has_proficiency[5] <= 1)
constraints.append(has_proficiency[6] <= 1)
constraints.append(has_proficiency[7] <= 1)
constraints.append(has_proficiency[8] <= 1)
constraints.append(has_proficiency[9] <= 1)
constraints.append(has_proficiency[10] <= 1)
constraints.append(has_proficiency[11] <= 1)
constraints.append(has_proficiency[12] <= 1)
constraints.append(has_proficiency[13] <= 1)
constraints.append(has_proficiency[14] <= 1)
constraints.append(has_proficiency[15] <= 1)
constraints.append(has_proficiency[16] <= 1)
constraints.append(has_proficiency[17] <= 1)

# Calculated ability scores
str = attributes[0] + racial_bonus[0]  + feat_bonus[0]
dex = attributes[1] + racial_bonus[1]  + feat_bonus[1]
con = attributes[2] + racial_bonus[2]  + feat_bonus[2]
int = attributes[3] + racial_bonus[3]  + feat_bonus[3]
wis = attributes[4] + racial_bonus[4]  + feat_bonus[4]
char = attributes[5] + racial_bonus[5]  + feat_bonus[5]

strmod = (str - 10) / 2
dexmod = (dex - 10) / 2
conmod = (con - 10) / 2
intmod = (int - 10) / 2
wismod = (wis - 10) / 2
charmod = (char - 10) / 2

#Skill weights determined by this website here: Low usefullness gets 0.2 mod, med gets 0.4, high gets 0.6, essential gets 0.8
# Use website here: https://gamerant.com/dungeons-dragons-best-worst-useful-ability-skills-ranked/#deception-charisma
damage_output = (((20 - (enemyAC - ((strengthScore - 10) / 2))) / 20) * ((avg_weapon_dmg[0] + 0.5) * ((strengthScore - 10) / 2)))
AC = 11 + attributes[1] + racial_bonus[1]  + feat_bonus[1]
hitpoints_per_level = 6 + attributes[2] + racial_bonus[2]  + feat_bonus[2]
skillweight = 0.6 * (dexmod  + proficiency_bonus * has_proficiency[0]) #acrobatics # 7
skillweight += 0.4 * (wismod + proficiency_bonus * has_proficiency[1]) #Animal handling # 14
skillweight += 0.4 * (intmod + proficiency_bonus * has_proficiency[2]) # Arcana # 11
skillweight += 0.6 * (strmod + proficiency_bonus * has_proficiency[3]) # Atheletics # 7 two number 7s
skillweight += 0.6 * (charmod + proficiency_bonus * has_proficiency[4]) # Deception # 6
skillweight += 0.4 * (intmod + proficiency_bonus * has_proficiency[5]) # History # 10
skillweight += 0.8 * (wismod + proficiency_bonus * has_proficiency[6]) # Insight # 4
skillweight += 0.4 * (charmod + proficiency_bonus * has_proficiency[7]) # Intimidation # 12
skillweight += 0.6 * (intmod + proficiency_bonus * has_proficiency[8]) # Investigation # 5
skillweight += 0.2 * (wismod + proficiency_bonus * has_proficiency[9]) # Medicine # 13
skillweight += 0.4 * (intmod + proficiency_bonus * has_proficiency[10]) # Nature 9
skillweight += 0.8 * (wismod + proficiency_bonus * has_proficiency[11]) # Perception 1
skillweight += 0.2 * (charmod + proficiency_bonus * has_proficiency[12]) #Performance high
skillweight += 0.8 * (charmod + proficiency_bonus * has_proficiency[13]) # Persuasion 2
skillweight += 0.2 * (intmod + proficiency_bonus * has_proficiency[14]) # Religion 17
skillweight += 0.6 * (dexmod + proficiency_bonus * has_proficiency[15]) #SOF 8
skillweight += 0.8 * (dexmod + proficiency_bonus * has_proficiency[16]) # stealth 3
skillweight += 0.2 * (wismod + proficiency_bonus * has_proficiency[17]) #survuval 15

passive_perception = 10 + wismod

obj_func= damage_output + AC + 2 * hitpoints_per_level + skillweight + passive_perception

                   
problem = cp.Problem(cp.Maximize(obj_func), constraints)

problem.solve(solver=cp.GUROBI,verbose = True)

print("obj_func =")
print(obj_func.value)
print("attributes =")
print(attributes.value)
print("avg_weapon_dmg =")
print(avg_weapon_dmg.value)
print("proficiencies =")
print(has_proficiency.value)
print("feat_bonus =")
print(feat_bonus.value)

                                     CVXPY                                     
                                     v1.3.2                                    
(CVXPY) Nov 29 05:11:52 PM: Your problem has 37 variables, 99 constraints, and 0 parameters.
(CVXPY) Nov 29 05:11:52 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Nov 29 05:11:52 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Nov 29 05:11:52 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Nov 29 05:11:52 PM: Compiling problem (target solver=GUROBI).
(CVXPY) Nov 29 05:11:52 PM: Reduction chain: FlipObjective -> CvxAttr2Constr -> Qp2SymbolicQp